In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from pandas.api.types import CategoricalDtype
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
def count_column_uniquevalues(df, c):
    return df.groupby(c).size()

# a. Estratégia de Modelagem

A tarefa abordada é a predição de `room_type`, para tal farei uso somente do dataset *Detailed Listings data*.

In [3]:
listings = pd.read_csv('listings.csv', low_memory=False)
pd.set_option('display.max_columns', None)
print(listings.shape)
listings.head(2)

(33715, 106)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,20191122092441,2019-11-22,Very Nice 2Br - Copacabana - WiFi,Pls note that special rates apply for Carnival...,- large balcony which looks out on pedestrian ...,Pls note that special rates apply for Carnival...,none,This is the best spot in Rio. Everything happe...,NaN,Excellent location. Close to all major public ...,The entire apartment is yours. It's like your ...,I will be available throughout your stay shoul...,Please leave the apartment in a clean fashion ...,NaN,NaN,https://a0.muscache.com/im/pictures/65320518/3...,NaN,68997,https://www.airbnb.com/users/show/68997,Matthias,2010-01-08,"Rio de Janeiro, State of Rio de Janeiro, Brazil",I am a journalist/writer. Lived for 15 years...,within an hour,100%,NaN,t,https://a0.muscache.com/im/pictures/user/67b13...,https://a0.muscache.com/im/pictures/user/67b13...,Copacabana,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Rio de Janeiro, Rio de Janeiro, Brazil",Copacabana,Copacabana,NaN,Rio de Janeiro,Rio de Janeiro,22020-050,Rio De Janeiro,"Rio de Janeiro, Brazil",BR,Brazil,-22.96592,-43.17896,t,Condominium,Entire home/apt,5,1.0,2.0,2.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,$332.00,NaN,NaN,$0.00,$378.00,2,$63.00,4,30,4,4,30,30,4.0,30.0,a week ago,t,23,31,35,303,2019-11-22,243,27,2010-07-15,2019-11-11,93.0,9.0,10.0,10.0,10.0,10.0,9.0,f,NaN,NaN,t,f,strict_14_with_grace_period,f,f,1,1,0,0,2.13
1,25026,https://www.airbnb.com/rooms/25026,20191122092441,2019-11-22,Beautiful Modern Decorated Studio in Copa,"Our apartment is a little gem, everyone loves ...",This newly renovated studio (last renovations ...,"Our apartment is a little gem, everyone loves ...",none,Copacabana is a lively neighborhood and the ap...,"For any stay superior to 15 days, an additiona...",At night we recommend you to take taxis only. ...,"internet wi-fi, cable tv, air cond, ceiling fa...","Only at check in, we like to leave our guests ...",Smoking outside only. Family building so pleas...,NaN,NaN,https://a0.muscache.com/im/pictures/3003965/68...,NaN,102840,https://www.airbnb.com/users/show/102840,Viviane,2010-04-03,"Rio de Janeiro, State of Rio de Janeiro, Brazil","Hi guys, We're a lovely team of 3 people:\r\n\...",within 

Dado que o dataset apresenta uma extensiva quantidade de colunas, 106 no total. Faço um curadoria pra avaliar o potencial da coluna como preditor e o tipo da variável, checando se precisará ou não de tratamentos como limpar string, transformar o tipo, ou aplicar one-hot encoding.

Não usarei colunas descritivas como `summary` ou `amenities`. Apesar que as mesmas podem conter informação relevante para o maior êxito na tarefa. Seria necessário um tempo maior de dedicação em cima delas, o que não é possível no momento.

Todas as colunas consideradas preditoras serão assinaladas em `listings_tidy`.

In [4]:
listings_tidy = pd.DataFrame()
listings_tidy['host_since_year'] = listings.host_since.apply(lambda x: np.float(str(x)[:4]))
listings_tidy['host_response_time'] = listings['host_response_time'].astype(CategoricalDtype( 
                                               categories=['within an hour', 'within a few hours', 'within a day',
                                                           'a few days or more'], ordered=True)).cat.codes
listings_tidy['host_response_rate'] = listings.host_response_rate.apply(lambda x: np.float(str(x).replace('%', ''))) 
listings_tidy['host_is_superhost'] = listings.host_is_superhost.apply(lambda x: 1 if x == 't' else 0)
listings_tidy['host_has_profile_pic'] = listings.host_is_superhost.apply(lambda x: 1 if x == 't' else 0)
listings_tidy['host_identity_verified'] = listings.host_is_superhost.apply(lambda x: 1 if x == 't' else 0)
listings_tidy['is_location_exact'] = listings.host_is_superhost.apply(lambda x: 1 if x == 't' else 0)

valid_PT = count_column_uniquevalues(listings, 'property_type').sort_values(ascending=False)[:10].index.values.tolist()
listings_tidy = pd.concat([listings_tidy, pd.get_dummies(listings['property_type'])[valid_PT]], axis=1)

valid_BT = count_column_uniquevalues(listings, 'bed_type').sort_values(ascending=False)[:2].index.values.tolist()
listings_tidy = pd.concat([listings_tidy, pd.get_dummies(listings['bed_type'])[valid_BT]], axis=1)

listings_tidy['price'] = listings.price.apply(lambda x: np.float(x[1:].replace(',', '')))
listings_tidy['weekly_price'] = listings.weekly_price.apply(lambda x: np.float(x[1:].replace(',', '')) if x == x else x)
listings_tidy['monthly_price'] = listings.monthly_price.apply(lambda x: np.float(x[1:].replace(',', '')) if x == x else x)
listings_tidy['security_deposit'] = listings.security_deposit.apply(lambda x: np.float(x[1:].replace(',', '')) if x == x else x)
listings_tidy['cleaning_fee'] = listings.cleaning_fee.apply(lambda x: np.float(x[1:].replace(',', '')) if x == x else x)
listings_tidy['extra_people'] = listings.extra_people.apply(lambda x: np.float(x[1:].replace(',', '')) if x == x else x)

listings_tidy['has_availability'] = listings.has_availability.apply(lambda x: 1 if x == 't' else 0)

listings_tidy['last_review_year'] = listings.last_review.apply(lambda x: np.float(str(x)[:4]))
listings_tidy['instant_bookable'] = listings.instant_bookable.apply(lambda x: 1 if x == 't' else 0)

valid_CP = count_column_uniquevalues(listings, 'cancellation_policy').sort_values(ascending=False)[:5].index.values.tolist()
listings_tidy = pd.concat([listings_tidy, pd.get_dummies(listings['cancellation_policy'])[valid_CP]], axis=1)

predictors_notreatment = ['host_listings_count', 'host_total_listings_count', 'latitude', 
                          'longitude', 'accommodates', 'bedrooms', 'beds', 'square_feet', 
                          'guests_included', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
                          'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
                          'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30', 
                          'availability_60', 'availability_90', 'availability_365', 'number_of_reviews',
                          'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy',
                          'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 
                          'review_scores_location', 'review_scores_value', 'calculated_host_listings_count',
                          'calculated_host_listings_count_entire_homes', 
                          'calculated_host_listings_count_private_rooms', 
                          'calculated_host_listings_count_shared_rooms', 
                          'reviews_per_month']
listings_tidy = pd.concat([listings_tidy, listings[predictors_notreatment]], axis=1)

Por último, eu adiciono a coluna com os rótulos já codificada de forma númerica para ser usado pelo algoritmo. E separo um parte das amostras para teste.

In [15]:
listings_tidy['room_type_cat'] = listings.room_type.astype('category').cat.codes
cat_labels = listings.room_type.astype('category').cat.categories.tolist()

dftrain = listings_tidy.sample(frac=0.8, random_state=200)
dftest = listings_tidy.drop(dftrain.index)
dftrain.head(2)

,host_since_year,host_response_time,host_response_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,is_location_exact,Apartment,House,Condominium,Serviced apartment,Loft,Guest suite,Bed and breakfast,Guesthouse,Villa,Hostel,Real Bed,Pull-out Sofa,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,has_availability,last_review_year,instant_bookable,flexible,strict_14_with_grace_period,moderate,super_strict_60,super_strict_30,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,square_feet,guests_included,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,room_type_cat
534,2012.0,1,100.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1998.0,NaN,NaN,0.0,400.0,0.0,1,2019.0,0,0,1,0,0,0,1.0,1.0,-23.00492,-43.30445,14,5.0,9.0,NaN,1,3,1125,3,3,1125,1125,3.0,1125.0,30,46,76,347,20,5,91.0,9.0,9.0,10.0,10.0,9.0,9.0,1,1,0,0,0.27,0
12203,2016.0,2,100.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,336.0,NaN,NaN,1050.0,210.0,0.0,1,NaN,1,0,1,0,0,0,3.0,3.0,-22.97033,-43.41687,5,3.0,3.0,NaN,1,15,1125,15,15,1125,1125,15.0,1125.0,30,60,90,365,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,0,1,NaN,0


É notável que o dataset é bem desigual entre as categorias de quarto. Pra compensar isso, eu atributo um peso a cada amostra. Como alguns pesos ficam absurdamente grandes, eu os ajustos a fim que não prejudique a correta classificação das classes de maior população.

In [8]:
count_column_uniquevalues(dftrain, 'room_type_cat')

room_type_cat
0    19180
1      244
2     6985
3      563
dtype: int64

In [9]:
roomtype_proportions = dftrain.groupby('room_type_cat').size().to_frame('weight').apply(lambda x: x.max() / x)
roomtype_proportions['weight_adj'] = roomtype_proportions.weight.apply(lambda x: x * np.exp(-x/100))
roomtype_proportions

,weight,weight_adj
room_type_cat,,
0,1.000000,0.990050
1,78.606557,35.815815
2,2.745884,2.671511
3,34.067496,24.231871


***
# b. Como foi definida a função de custo utilizada?

Como a tarefa é uma classificação de multiclasses. A função custo usada é a *cross-entropy loss*.

\begin{align}
L & = -\sum_{i}^{C}y_ilog(\hat{y_i})
\end{align}

Para gradiente boosting, é possível usar qualquer função custo, passando para o algoritmo, apenas o gradiente e o hessian da função. Na tarefa aqui, não precisei obviamente, por se tratar de uma função custo já *default*. Detalhes da implementação podem ser vistos [aqui](https://github.com/microsoft/LightGBM/blob/master/src/objective/multiclass_objective.hpp#L81).

***
# Por que escolheu utilizar este método?

Para modelagem, usarei o algoritmo de *gradient boosting*, mais especificamente o implementado no módulo lightgbm. *Boosting* é uma meta-algoritmo que aplica uma série de modelos fracos (de baixa performance) que somados representarão um modelo forte (de alta performance). E no caso, o termo gradiente denota o fato que o modelo seguinte aprende a partir do gradiente resultante do somatório dos modelos anteriores. 

As implementações atuais de *Grandient Boosting* são simples de operar, rápidas, e normalmente performam melhor que os outros algoritmos para tarefas genéricas / analíticas de classificação e regressão tanto para modelagem linear como não-linear. 

Outra vantagem do algoritmo, quando usado com árvores de decisão, é ele funcionar como seletor de variáveis. Por isso, adicionei todas variáveis do dataset original que podem ser úteis, não me preocupando com correlação entre as variáveis ou variáveis que são "ruídos". O algoritmo decide que variáveies são mais importantes para a realização da tarefa.

Escolhi *lightgbm* pela velocidade de treinamento. Principalmente pela quantidade de colunas-variáveis que estou lidando, 69 ao todo. *Lightgbm* tem as qualidades de *Exclusive Feature Bundling* que acelera o treinamento quando há muitas colunas esparsas de one-hot-encoding, e o fato de seguir leaf-wise no crescimento das árvores também pode acelerar o treinamento.

***
# c. Qual foi o critério utilizado na seleção do modelo final? / d. Qual foi o critério utilizado para validação do modelo?

Para validar o modelo, usei k-fold Cross Validation (CV), no caso **k** = 3. O uso de valor pequeno para **k** garante que os *folds* tenham uma maior variação entre si. Usando a métrica de erro de CV para apontar um limite contra overfitting. Pois também uso o k-cv para achar um número máximo de iterações do algoritmo antes começar a apresentar variância.

In [10]:
dm_train = lgb.Dataset(dftrain.drop('room_type_cat', axis=1), label=dftrain['room_type_cat'],
                       weight=dftrain.merge(roomtype_proportions, how='left', on='room_type_cat')['weight_adj'])

params = {'boosting_type': 'gbdt',
          'objective': 'softmax',
          'num_classes': 4,
          'enable_bundle': True,
          'max_depth': 10,
          'min_split_gain': 10,
          'lambda_l2': 1,
          'lambda_l1': 1,
          'subsample': .75,
          'subsample_freq': 1,
          'feature_fration': .5,
          'metric': ['multi_error'],
          'learning_rate': 0.09}

In [11]:
cv_hist = lgb.cv(params,
                 dm_train,
                 verbose_eval=25,
                 num_boost_round=20000,
                 nfold=3,
                 early_stopping_rounds=50,
                 show_stdv=False)  

[25]	cv_agg's multi_error: 0.0315299
[50]	cv_agg's multi_error: 0.0305286
[75]	cv_agg's multi_error: 0.0306813


In [12]:
best_iter = np.argmin(cv_hist['multi_error-mean'])
print('best iteration: {}'.format(best_iter))

gbm_model = lgb.train(params,
                      dm_train,
                      verbose_eval=25,
                      num_boost_round=best_iter,
                      valid_sets=dm_train)                   

best iteration: 41
[25]	training's multi_error: 0.0233979


***
# e. Quais evidências você possui de que seu modelo é suficientemente bom?

A evidência baseia-se na performance sobre os 20% das amostragens aleatórias (quase 7000) retiradas do dataset original. Portanto, num cenário de produção, espera-se que a performance do sistema seja similar a essa se as amostras de produção pertecerem a mesma distribuição que as amostras de teste.

In [13]:
preds_train = gbm_model.predict(dftrain)
preds_train = np.argmax(preds_train, axis=1)
print('train overall accuracy: {0:.2f}'.format(np.mean(preds_train == dftrain.room_type_cat)))
print('\nConfusion Matrix:')
print(confusion_matrix(preds_train, dftrain['room_type_cat']))
print('\n')
print(classification_report(preds_train, dftrain['room_type_cat'], target_names=cat_labels))

train overall accuracy: 0.97

Confusion Matrix:
[[18681     0    74     0]
 [   92   242     7     0]
 [  355     1  6751     1]
 [   52     1   153   562]]


                 precision    recall  f1-score   support

Entire home/apt       0.97      1.00      0.98     18755
     Hotel room       0.99      0.71      0.83       341
   Private room       0.97      0.95      0.96      7108
    Shared room       1.00      0.73      0.84       768

       accuracy                           0.97     26972
      macro avg       0.98      0.85      0.90     26972
   weighted avg       0.97      0.97      0.97     26972



In [14]:
preds_test = gbm_model.predict(dftest)
preds_test = np.argmax(preds_test, axis=1)
print('test overall accuracy: {0:.2f}'.format(np.mean(preds_test == dftest.room_type_cat)))
print('\nConfusion Matrix:')
confusion_matrix(preds_test, dftest['room_type_cat'])
print(classification_report(preds_test, dftest['room_type_cat'], target_names=cat_labels))

test overall accuracy: 0.97

Confusion Matrix:
                 precision    recall  f1-score   support

Entire home/apt       0.97      0.99      0.98      4651
     Hotel room       0.92      0.69      0.79        95
   Private room       0.97      0.94      0.95      1794
    Shared room       0.99      0.74      0.85       203

       accuracy                           0.97      6743
      macro avg       0.96      0.84      0.89      6743
   weighted avg       0.97      0.97      0.97      6743



Analisando f1-score das classes entre o conjunto de treinamento e teste, vemos que o modelo possui baixíssima variância, generalizando bem. E tento uma métrica razoável entre todas as classes. As classes de menor frequência como `Hotel room` e `Shared room` tiveram recall razoavelmente abaixo das demais, contudo, é uma perda esperada ponderando a acurácia geral do sistema.